In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
import glob
import pandas as pd 
from natsort import natsorted

In [2]:
image_size = (180, 180)
channels = 3
#본 모델에서 180, 180 3채널로 고정시켰기 때문에 고정변수입니다 이미지사이즈

In [3]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
exposure_model = load_model('/content/drive/MyDrive/ttproject/model/newVGG16(노출비노출).h5')
violence_model = load_model('/content/drive/MyDrive/ttproject/model/newVGG16(폭력비폭력).h5')
relationship_model = load_model('/content/drive/MyDrive/ttproject/model/newVGG16(관계비관계).h5')

In [5]:
naked_classes = ['Naked', 'Non-Naked'] 
violence_classes = ['Non-Violence', 'Violence'] 
sexual_classes = ['Sexual', 'Non-Sexual'] 

In [8]:
def preprocess_image(image):
  image = cv2.resize(image, image_size)
  image = image.astype('float32') / 255.0
  return image

In [9]:
def classify_image(image):
    preprocessed_image = preprocess_image(image)
    naked_prediction = exposure_model.predict(np.expand_dims(preprocessed_image, axis=0))[0]
    violence_prediction = violence_model.predict(np.expand_dims(preprocessed_image, axis=0))[0]
    sexual_prediction = relationship_model.predict(np.expand_dims(preprocessed_image, axis=0))[0]

    # 확률이 0.6 이상일 경우 해당 클래스를 반환, 그렇지 않으면 'Uncertain' 반환
    naked_class = naked_classes[np.argmax(naked_prediction)] if np.max(naked_prediction) > 0.6 else 'Uncertain'
    violence_class = violence_classes[np.argmax(violence_prediction)] if np.max(violence_prediction) > 0.6 else 'Uncertain'
    sexual_class = sexual_classes[np.argmax(sexual_prediction)] if np.max(sexual_prediction) > 0.6 else 'Uncertain'

    return naked_class, violence_class, sexual_class

In [15]:
path="/content/drive/MyDrive/ttproject/tiktok_video/건전영상/건전영상044"
filenames = natsorted(os.listdir(path))

test_df = pd.DataFrame({"filename":filenames})
test_df["path"] = test_df["filename"].apply(lambda x: os.path.join(path, x))
test_df[["노출", "폭력", "관계"]] = test_df["path"].apply(lambda x: pd.Series(classify_image(cv2.imread(x))))
test_df = test_df.drop(columns=['path'])

test_df

1/1 [==============================] - 0s 21ms/step


,filename,노출,폭력,관계
0,frame_0.jpg,Non-Naked,Non-Violence,Non-Sexual
1,frame_1.jpg,Non-Naked,Non-Violence,Non-Sexual
2,frame_2.jpg,Non-Naked,Non-Violence,Non-Sexual
3,frame_3.jpg,Non-Naked,Non-Violence,Sexual
4,frame_4.jpg,Uncertain,Non-Violence,Sexual
5,frame_5.jpg,Non-Naked,Non-Violence,Non-Sexual
6,frame_6.jpg,Uncertain,Non-Violence,Non-Sexual
7,frame_7.jpg,Non-Naked,Non-Violence,Non-Sexual
8,frame_8.jpg,Non-Naked,Non-Violence,Non-Sexual
9,frame_9.jpg,Non-Naked,Non-Violence,Sexual
